# Imports and setup

In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["EEG_ROOT"] = "/home/ms994"
import util_funcs
from importlib import reload
reload(util_funcs)
from copy import deepcopy as cp

import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import pywt
import tsfresh.feature_extraction.feature_calculators as feats
import constants
import clinical_text_analysis as cta
reload(cta)
import tsfresh
import ensembleReader as er
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, r2_score
from os import path

reload(read)
import wf_analysis.datasets as wfdata
import wf_analysis.filters as filt
reload(filt)
from addict import Dict
import time
import pickle as pkl
import tsfresh.feature_extraction.feature_calculators as tsf
import pyedflib as edf
import seaborn as sns
import multiprocessing as mp
import numpy.random as random
import statsmodels

Using Theano backend.


In [2]:
reader = read.EdfDataset("train", "01_tcp_ar", use_numpy=True, n_process=2)

# List of ideas to try out
Longer sequences. It appears that human observers are looking at up to 10+ minutes of EEG data at a time, im only doing 4 second windows in isolation.

See first with hand engineered FFT and coherence features

Also see if i can't include sequences of outputs i.e. t

## Try just doing FNSZ and GNSZ seizure types

## Long sequence

### FFT + Coherence

In [10]:
fft = read.EdfFFTDatasetTransformer(reader, is_pandas_data=False, freq_bins=constants.FREQ_BANDS, window_size=pd.Timedelta(seconds=0.5), )

In [13]:
fftData = fft[0][0]

In [14]:
fftData.shape

(21, 2610, 4)

In [17]:
coherenceReader = wfdata.CoherenceTransformer(reader, is_pandas=False)

### labels

In [3]:
%%bash
cat /home/ms994/v1.5.0/edf/dev_test/01_tcp_ar/002/00000258/s002_2003_07_21/00000258_s002_t002.lbl


version = lbl_v1.0.0

montage = 0, FP1-F7: EEG FP1-REF --  EEG F7-REF
montage = 1, F7-T3:  EEG F7-REF  --  EEG T3-REF
montage = 2, T3-T5:  EEG T3-REF  --  EEG T5-REF
montage = 3, T5-O1:  EEG T5-REF  --  EEG O1-REF
montage = 4, FP2-F8: EEG FP2-REF --  EEG F8-REF
montage = 5, F8-T4:  EEG F8-REF  --  EEG T4-REF
montage = 6, T4-T6:  EEG T4-REF  --  EEG T6-REF
montage = 7, T6-O2:  EEG T6-REF  --  EEG O2-REF
montage = 8, A1-T3:  EEG A1-REF  --  EEG T3-REF
montage = 9, T3-C3:  EEG T3-REF  --  EEG C3-REF
montage = 10, C3-CZ:  EEG C3-REF  --  EEG CZ-REF
montage = 11, CZ-C4:  EEG CZ-REF  --  EEG C4-REF
montage = 12, C4-T4:  EEG C4-REF  --  EEG T4-REF
montage = 13, T4-A2:  EEG T4-REF  --  EEG A2-REF
montage = 14, FP1-F3: EEG FP1-REF --  EEG F3-REF
montage = 15, F3-C3:  EEG F3-REF  --  EEG C3-REF
montage = 16, C3-P3:  EEG C3-REF  --  EEG P3-REF
montage = 17, P3-O1:  EEG P3-REF  --  EEG O1-REF
montage = 18, FP2-F4: EEG FP2-REF --  EEG F4-REF
montage = 19, F4-C4:  EEG F4-REF  --  EEG C4-REF
montage

In [47]:
fn = "/home/ms994/v1.5.0/edf/dev_test/01_tcp_ar/002/00000258/s002_2003_07_21/00000258_s002_t002.edf"
fn[:-4]+".lbl"


'/home/ms994/v1.5.0/edf/dev_test/01_tcp_ar/002/00000258/s002_2003_07_21/00000258_s002_t002'

##### TODO
check that each lbl file has only one level
check that each lbl file is consistent with the tse file

In [56]:
fn = "/home/ms994/v1.5.0/edf/dev_test/01_tcp_ar/002/00000258/s002_2003_07_21/00000258_s002_t002.lbl"
def get_per_channel_annotation(fn):
    with open(fn, "rb") as  open_label_file:
        output = open_label_file.readlines()
    montage_channel_dict = {}
    start_montage_channel_assign = False
    symbols = [None]
    symbols_list = []
    full_list = []
    for line in output:
        if "montage = 0" in str(line):
            start_montage_channel_assign = True
        if start_montage_channel_assign:
            if b'\n' == (line):
                start_montage_channel_assign = False
            else:
                montage_channel_dict[int(line[10:str(line).index(',') - 2])] = (line[str(line).index(','):str(line).index(':') - 2]).decode("utf-8") 
        elif "symbols[0]" in str(line):
            #since we are using weird pseudocode, run the string through exec and get the symbols variable out
            exec(line.decode("utf8"))
            symbols_dict = symbols
            symbols_list = [symbols_dict[0][i] for i in range(len(symbols[0].keys()))]
    #         print(symbols_list)
        elif "label =" in str(line):
#             print(line.decode("utf8").replace("{", "[").replace("}", "]"))
            #since we are using weird pseudocode, run the string through exec and get the label variable out
            label = eval(line.decode("utf8").replace("{", "[").replace("}", "]")[7:])
            
            curr_level = label[0]
            curr_sublevel = label[1]
            curr_start = label[2]
            curr_end = label[3]
            curr_mont_num = label[4]
            curr_channel = montage_channel_dict[int(curr_mont_num)]
            curr_label_probs = label[5]
            curr_vals = pd.Series([curr_level, curr_sublevel, curr_start, curr_end, curr_channel, *curr_label_probs], index=["level", "sublevel", "start", "end", "channel", *symbols_list])
            full_list.append(curr_vals)
#             del label


    return pd.concat(full_list, axis=1).T

In [84]:
data = get_per_channel_annotation(fn)
data

,level,sublevel,start,end,channel,(null),spsw,gped,pled,eyem,...,nesz,intr,slow,eyem,chew,shiv,musc,elpp,elst,calb
0,0,0,0,22.0375,FP1-F7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,22.0375,136.438,FP1-F7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,136.438,272,FP1-F7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,22.0375,F7-T3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,22.0375,136.438,F7-T3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,136.438,272,F7-T3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,22.0375,T3-T5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,22.0375,136.438,T3-T5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,136.438,272,T3-T5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,22.0375,T5-O1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
def gen_seizure_channel_labels(fn, width=pd.Timedelta(seconds=0.5)):
    data = get_per_channel_annotation(fn)
    max_time = pd.Timedelta(seconds=int(data.end.max()))
    index = pd.timedelta_range(start=pd.Timedelta(0), end=max_time, freq=width)
    columns = data.channel.unique()
    expanded_is_seizure = pd.DataFrame(index=index, columns=columns).fillna(0)
    for i, row in data.iterrows():
        expanded_is_seizure.loc[pd.Timedelta(seconds=row.start):pd.Timedelta(seconds=row.end), row.channel] = 1 - row.bckg
    return expanded_is_seizure

In [79]:
edf_files = read.get_all_token_file_names("train", "01_tcp_ar")
reload(read)
lbl_files = [read.get_associated_lbl(fn) for fn in edf_files]

In [80]:
all_channels = set()
len_channels = set()
for lbl_fn in lbl_files:
    all_channels = all_channels.union(read.get_per_channel_annotation(lbl_fn).channel.unique())
    len_channels.add(len(read.get_per_channel_annotation(lbl_fn).channel.unique()))

In [81]:
edf_files = read.get_all_token_file_names("dev_test", "01_tcp_ar")
reload(read)
lbl_files = [read.get_associated_lbl(fn) for fn in edf_files]
for lbl_fn in lbl_files:
    all_channels = all_channels.union(read.get_per_channel_annotation(lbl_fn).channel.unique())
    len_channels.add(len(read.get_per_channel_annotation(lbl_fn).channel.unique()))

In [82]:
list(all_channels)

['C3-CZ',
 'CZ-C4',
 'FP1-F7',
 'F8-T4',
 'F7-T3',
 'C4-T4',
 'FP2-F4',
 'T5-O1',
 'C4-P4',
 'F3-C3',
 'T3-T5',
 'A1-T3',
 'T4-A2',
 'FP2-F8',
 'FP1-F3',
 'T3-C3',
 'C3-P3',
 'T4-T6',
 'P4-O2',
 'P3-O1',
 'T6-O2',
 'F4-C4']

In [83]:
list(len_channels)

[22]

In [53]:
util_funcs.get_common_channel_names()

['EEG T5-REF',
 'EEG C3-REF',
 'EEG A1-REF',
 'EEG P4-REF',
 'EEG F8-REF',
 'EEG F3-REF',
 'EEG CZ-REF',
 'EEG O2-REF',
 'EEG A2-REF',
 'EEG P3-REF',
 'EEG C4-REF',
 'EEG O1-REF',
 'EEG T4-REF',
 'EEG F4-REF',
 'EEG F7-REF',
 'EEG PZ-REF',
 'EEG FP1-REF',
 'EEG FZ-REF',
 'EEG T3-REF',
 'EEG T6-REF',
 'EEG FP2-REF']

In [58]:
gen_seizure_channel_labels(fn).columns

Index(['FP1-F7', 'F7-T3', 'T3-T5', 'T5-O1', 'FP2-F8', 'F8-T4', 'T4-T6',
       'T6-O2', 'A1-T3', 'T3-C3', 'C3-CZ', 'CZ-C4', 'C4-T4', 'T4-A2', 'FP1-F3',
       'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2'],
      dtype='object')

In [27]:
data_x = reader[0][0]

In [34]:
constants.COMMON_DELTA

0.004

### Data Binning

In [3]:
def time_distribute_x(data_x, width=pd.Timedelta(seconds=4), stride=pd.Timedelta(seconds=1)):
    time_steps = []
    literal_width = int(width/pd.Timedelta(seconds=constants.COMMON_DELTA))
    literal_stride = int(stride/pd.Timedelta(seconds=constants.COMMON_DELTA))
    for i in range(int(data_x.shape[0]/literal_stride - width/stride)):
        time_steps.append(data_x[i*literal_stride:(i) * literal_stride + literal_width])
    time_steps = np.array(time_steps)
    return time_steps

In [51]:
time_distribute_x(data_x).shape

(1301, 1000, 21)

In [15]:
len(reader.edf_tokens)

3177

In [ ]:
modify_self()

In [67]:
class LSTMDataset(util_funcs.MultiProcessingDataset):
    def __init__(self, reader, width=pd.Timedelta(seconds=4), stride=pd.Timedelta(seconds=2)):
        self.width = width
        self.stride = stride
        self.reader = reader
    def __len__(self):
        return len(self.reader)
    def __getitem__(self, i):
        if self.should_use_mp(i):
            return self.getItemSlice(i)
        data_x, labels = self.reader[i]
        data_x = read.time_distribute_x(data_x, width=self.width, stride=self.stride)
        return data_x, read.expand_tse_file_seizure_only(labels, time_period=self.stride).iloc[0:data_x.shape[0]]
        

In [68]:
reload(read)
lstm_data = LSTMDataset(reader)

In [69]:
reader[10][0].shape

(150240, 21)

In [70]:
150240/(250)/60

10.016

In [71]:
dp = lstm_data[10]

In [73]:
dp[0].shape

(298, 1000, 21)

In [9]:
import sys

sys.getsizeof(data[0])

218568128

In [13]:
stride = lstm_data.stride
labels = reader[0][1]
index = pd.timedelta_range(start=pd.Timedelta(0), end=pd.Timedelta(seconds=labels.end.max()), freq=stride)


In [27]:
reload(read)
seiz_series = read.expand_tse_file_seizure_only(labels)

In [28]:
seiz_series

,0,1
00:00:00,True,False
00:00:01,True,False
00:00:02,True,False
00:00:03,True,False
00:00:04,True,False
00:00:05,True,False
00:00:06,True,False
00:00:07,True,False
00:00:08,True,False
00:00:09,True,False


In [10]:
util_funcs.get_file_sizes("train", "01_tcp_ar").mz

,1
0,
/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013182/s001_2015_10_07/00013182_s001_t000.edf,1305
/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013145/s007_2015_09_04/00013145_s007_t008.edf,601
/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013145/s007_2015_09_04/00013145_s007_t002.edf,300
/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013145/s007_2015_09_04/00013145_s007_t003.edf,601
/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013145/s007_2015_09_04/00013145_s007_t000.edf,300
/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013145/s007_2015_09_04/00013145_s007_t009.edf,300
/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013145/s007_2015_09_04/00013145_s007_t007.edf,300
/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013145/s007_2015_09_04/00013145_s007_t004.edf,601
/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013145/s007_2015_09_04/00013145_s007_t005.edf,300
